## Importing libs

In [1]:
import time
import numpy as np
import pandas as pd
import pickle

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import json

### Set up selenium browser

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('headless')    
service = Service('./chromedriver')
driver = webdriver.Chrome(service=service, options=options)

## Web scraping TASS

In [4]:
def get_content(driver: webdriver, api: str) -> json:
    '''Open a link by selenium webdriver, scrap the page and returns result with proposed news in json format'''
    driver.get(api) # открываем ссылку
    time.sleep(3) # даем прогрузиться
    
    page_source = driver.page_source # берем страницу
    soup = BeautifulSoup(page_source, 'html.parser') # парсим
    data = json.loads(soup.body.text) # достаем текст и превращаем в формат json
    
    return data


def update_news(news: list, data: json):
    '''Download news from Tass ural (/ural-news) and add to the existing list of news
    '''
    new_news = [] # список всех распарсенных новостей
    for item in data['result']: # идем по каждой новости
        cur_news = {} 
        cur_news['composite_id'] = item['composite_id']
        cur_news['title'] = item['title']
        cur_news['subtitle'] = item['subtitle']
        cur_news['lead'] = item['lead']
        cur_news['url'] = 'https://tass.ru' + item['url']
        cur_news['rubrics'] = []
        for rubric in item['rubrics']:
            cur_news['rubrics'].append(rubric['name'])
        cur_news['tags'] = []
        for tag in item['tags']:
            cur_news['tags'].append(tag['name'])
        cur_news['es_updated_dt'] = item['es_updated_dt']
        cur_news['published_date'] = item['es_updated_dt'].split('T')[0]
        new_news.append(cur_news)
        
    news.extend(new_news) # расширяем существующий список новыми новостями
    pass

In [5]:
# загружаем новости со стартовой страницы
api = 'https://tass.ru/tbp/api/v1/content?limit=18&offset=0&lang=ru&rubrics=/ural-news&sort=-es_updated_dt'
data = get_content(driver, api)

In [6]:
data['result'][0] # так выглядит новость

{'id': 14976661,
 'composite_id': '14976661',
 'type': 'news',
 'status': 'published',
 'title': 'Грозы с сильным ветром и жаркая погода ожидаются в регионах Урала в ближайшие три дня',
 'subtitle': '',
 'lead': 'Синоптики прогнозируют грозы с сильным ветром и дожди',
 'url': '/obschestvo/14976661',
 'lang': 'ru',
 'urgency': 'normal',
 'main_media': {'gallery': None, 'image': None, 'video': None},
 'main_rubric_id': 24,
 'rubrics': [{'id': 24,
   'name': 'Общество',
   'url': '/obschestvo',
   'type': 'rubric'},
  {'id': 4731,
   'name': 'Новости Урала',
   'url': '/ural-news',
   'type': 'rubric'}],
 'tags': [{'id': 3929, 'name': 'Россия', 'url': '/rossiya'},
  {'id': 8974,
   'name': 'Свердловская область',
   'url': '/sverdlovskaya-oblast'}],
 'published_dt': '2022-06-20T13:53:40',
 'publish_updated_dt': '2022-06-20T13:53:40',
 'updated_dt': '2022-06-20T13:53:40',
 'es_updated_dt': '2022-06-20T13:53:40.775132',
 'meta_title': 'Грозы с сильным ветром и жаркая погода ожидаются в реги

In [8]:
news[0] # так выглядит каждая новость после парсинга

{'composite_id': '14976661',
 'title': 'Грозы с сильным ветром и жаркая погода ожидаются в регионах Урала в ближайшие три дня',
 'subtitle': '',
 'lead': 'Синоптики прогнозируют грозы с сильным ветром и дожди',
 'url': 'https://tass.ru/obschestvo/14976661',
 'rubrics': ['Общество', 'Новости Урала'],
 'tags': ['Россия', 'Свердловская область'],
 'es_updated_dt': '2022-06-20T13:53:40.775132',
 'published_date': '2022-06-20'}

## Downloading past news

In [2]:
with open('tass', 'rb') as f:
    news = pickle.load(f)

In [26]:
i = 0
while True:
    time.sleep(3)
    i += 1
    last_es_updated_dt = news[-1]['es_updated_dt']
    iter_api = f'https://tass.ru/tbp/api/v1/content?limit=15&last_es_updated_dt={last_es_updated_dt}&lang=ru&rubrics=/ural-news&sort=-es_updated_dt'
    data = get_content(driver, iter_api)
    update_news(news, data)
    if i == 100:
        print('Reached 100!')
        i = 0
        with open('tass', 'wb') as f:
            pickle.dump(news, f)

Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!
Reached 100!


KeyboardInterrupt: 

In [ ]:
driver.close()

In [18]:
with open('tass', 'wb') as f:
    pickle.dump(news, f)

In [3]:
len(news)

34321

In [4]:
news[-1]

{'composite_id': '697849',
 'title': 'Лосенок, которого спасли на Урале, первые в жизни морозы перезимует в конюшне',
 'subtitle': '',
 'lead': 'Подкидыша выходили свердловские школьники',
 'url': 'https://tass.ru/ural-news/697849',
 'rubrics': ['Новости Урала'],
 'tags': [],
 'es_updated_dt': '2013-10-29T10:54:39.461413',
 'published_date': '2013-10-29'}

## Working with news data

In [15]:
df = pd.DataFrame.from_records(news)

In [16]:
df_econ = df[df['url'].apply(lambda x: 'ekonomika' in x)]
df_econ.reset_index(drop=True, inplace=True)

In [21]:
df_econ

,composite_id,title,subtitle,lead,url,rubrics,tags,es_updated_dt,published_date
0,14974687,Удмуртия заключила на ПМЭФ 10 соглашений и мем...,,В республике планируется создать около 500 нов...,https://tass.ru/ekonomika/14974687,"[Экономика и бизнес, Новости Урала, ПМЭФ-2022]","[Россия, Санкт-Петербург, Удмуртия, ПМЭФ-2022]",2022-06-20T11:50:32.453984,2022-06-20
1,14973665,Энергоблок №4 Белоярской АЭС вывели в плановый...,,После окончания работ объект будет работать на...,https://tass.ru/ekonomika/14973665,"[Экономика и бизнес, Новости Урала]","[Россия, Свердловская область]",2022-06-20T10:44:11.490726,2022-06-20
2,14969993,"ЯНАО, Москва и Магаданская область возглавили ...",,Всего в рейтинг вошел 41 регион,https://tass.ru/ekonomika/14969993,"[Общество, Экономика и бизнес, Северо-Запад, Н...",[Россия],2022-06-20T00:39:00.676491,2022-06-20
3,14966253,Золотодобывающее предприятие в Таджикистане по...,,Торговый представитель РФ в Таджикистане Евген...,https://tass.ru/ekonomika/14966253,"[Экономика и бизнес, Новости Урала]","[Таджикистан, Россия]",2022-06-18T19:26:17.044375,2022-06-18
4,14964641,"В Кировской области заявили, что перспективы о...",,По словам врио губернатора Александра Соколова...,https://tass.ru/ekonomika/14964641,"[Экономика и бизнес, Новости Урала, ПМЭФ-2022]","[Россия, Кировская область, ПМЭФ-2022]",2022-06-18T13:42:21.915387,2022-06-18
...,...,...,...,...,...,...,...,...,...
3204,1457112,"Инвестиционный форум ""Сочи-2014"": российские р...",,"Главы трех регионов заявили, что прекращение и...",https://tass.ru/ekonomika/1457112,"[Экономика и бизнес, Новости Урала, Санкции в ...","[Санкции в отношении России, Международный инв...",2014-09-20T17:00:09.896800,2014-09-20
3205,1451220,Югра покажет 3D-макет Всемирного торгового цен...,,Всего в Сочи Югра представит проекты на общую ...,https://tass.ru/ekonomika/1451220,"[Экономика и бизнес, Новости Урала, Международ...",[Международный инвестфорум в Сочи],2014-09-18T12:33:23.947044,2014-09-18
3206,1391961,Роспотребнадзор расширяет географию проверок с...,,"Ведомство начало проверку в Московской, Свердл...",https://tass.ru/ekonomika/1391961,"[Экономика и бизнес, Потребительский рынок, Но...",[Проблемы McDonald's в России],2014-08-21T09:11:03.309157,2014-08-21
3207,1290171,"""Добролет"" открывает продажу билетов на ежедне...",,Минимальный тариф составит от 999 руб.,https://tass.ru/ekonomika/1290171,"[Экономика и бизнес, Новости Урала, Транспорт]",[],2014-07-01T07:42:37.620436,2014-07-01
